In [1]:
pip install requests beautifulsoup4 faiss-cpu numpy sentence-transformers transformers chromadb


In [2]:
import requests
from bs4 import BeautifulSoup
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json
from transformers import pipeline
import re
import os
import chromadb

# Initialize models
model = SentenceTransformer("all-MiniLM-L6-v2")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

BASE_URL = "https://devgan.in"
LAW_TYPE = "bns"

# FAISS setup
d = 384  # Dimension of embeddings
index = faiss.IndexFlatL2(d)

# ChromaDB setup
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="bns_laws")

# Ensure output directory exists
output_dir = "./Legal_AEye-Opener"
os.makedirs(output_dir, exist_ok=True)

# Single JSON storage
all_laws_data = []

def extract_sections(content):
    """Extracts sections while keeping IPC references inside BNS sections."""
    sections = []
    section_pattern = re.split(r"(?=Section\s+\d+\s*–)", content)  # Match "Section X –" format

    for sec in section_pattern:
        match = re.match(r"Section\s+(\d+)\s*–\s*(.*)", sec.strip())
        if match:
            section_number = match.group(1)
            section_title = match.group(2)
            section_content = sec.strip()
        else:
            continue  # Skip if it doesn't match a section

        sections.append({
            "section_number": section_number,
            "section_title": section_title,
            "section_content": section_content
        })

    return sections

def summarize_content(content):
    """Summarizes the given content."""
    return summarizer(content[:1024], max_length=150, min_length=50, do_sample=False)[0]['summary_text']

# Fetch the BNS chapters
MAIN_URL = f"{BASE_URL}/{LAW_TYPE}/"
response = requests.get(MAIN_URL)
soup = BeautifulSoup(response.text, "html.parser")

chapters = []
for row in soup.select("table.menu tr"):
    columns = row.find_all("td")
    if len(columns) == 2:
        chapter_number = columns[0].text.strip()
        chapter_title = columns[1].text.strip()
        chapter_link = BASE_URL + columns[1].find("a")["href"]
        chapters.append((chapter_number, chapter_title, chapter_link))

# Scrape each chapter and store in a single JSON
for chapter_number, chapter_title, chapter_link in chapters:
    print(f"📥 Fetching Chapter {chapter_number}: {chapter_title}")

    chapter_response = requests.get(chapter_link)
    chapter_soup = BeautifulSoup(chapter_response.text, "html.parser")

    content_div = chapter_soup.find("div", id="content")
    if not content_div:
        print(f"⚠️ Warning: Could not find content div in Chapter {chapter_number}. Trying fallback method...")
        content_div = chapter_soup.find("div", class_="main-content")

    chapter_content = content_div.get_text(separator="\n", strip=True) if content_div else "Content not found."

    # Extract sections
    sections = extract_sections(chapter_content)

    for section in sections:
        section_summary = summarize_content(section["section_content"])
        formatted_summary = f"Section {section['section_number']}: {section_summary}" if section["section_number"] != "Unknown" else section_summary

        # Generate embedding
        embedding = model.encode(section["section_content"]).astype(np.float32)
        index.add(np.array([embedding]))  # Add to FAISS

        # Store in ChromaDB
        collection.add(
            ids=[f"{LAW_TYPE}_{chapter_number}_{section['section_number']}"],
            documents=[section["section_content"]],
            metadatas=[{
                "law_type": LAW_TYPE.upper(),
                "chapter_number": chapter_number,
                "chapter_title": chapter_title,
                "section_number": section["section_number"],
                "section_title": section["section_title"],
                "section_summary": formatted_summary,
                "source_url": chapter_link
            }]
        )

        # Append to single JSON data
        all_laws_data.append({
            "law_type": LAW_TYPE.upper(),
            "chapter_number": chapter_number,
            "chapter_title": chapter_title,
            "section_number": section["section_number"],
            "section_title": section["section_title"],
            "section_content": section["section_content"],
            "section_summary": formatted_summary,
            "source_url": chapter_link
        })

        print(f"✅ Processed Chapter {chapter_number}, Section {section['section_number']}")

# Save all data to a single JSON file
json_output_path = os.path.join(output_dir, "bns_laws_data.json")
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(all_laws_data, f, ensure_ascii=False, indent=4)

# Save FAISS index (backup)
faiss.write_index(index, os.path.join(output_dir, "bns_faiss.index"))

print(f"🎉 BNS Laws Scraping Completed! Data saved in: {json_output_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


📥 Fetching Chapter 01: Preliminary


✅ Processed Chapter 01, Section 1


✅ Processed Chapter 01, Section 2


✅ Processed Chapter 01, Section 3
📥 Fetching Chapter 02: Punishments


...

✅ Processed Chapter 19, Section 354


✅ Processed Chapter 19, Section 355


✅ Processed Chapter 19, Section 356


✅ Processed Chapter 19, Section 357
📥 Fetching Chapter 20: Repeal And Savings


✅ Processed Chapter 20, Section 358
🎉 BNS Laws Scraping Completed! Data saved in: ./Legal_AEye-Opener/bns_laws_data.json


In [1]:
import chromadb

# Connect to ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="bns_laws")

# Check stored documents
documents = collection.get()

if documents and "ids" in documents and len(documents["ids"]) > 0:
    print(f"✅ Data is stored in ChromaDB. Total records: {len(documents['ids'])}")
else:
    print("❌ No data found in ChromaDB.")


✅ Data is stored in ChromaDB. Total records: 358


In [7]:
from sentence_transformers import SentenceTransformer
import chromadb

# Load Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Connect to ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="bns_laws")

# Query ChromaDB
query = "murder"
query_embedding = model.encode(query).tolist()

# Search ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5  # Return top 5 matches
)

# Check if results exist
if results and results.get('documents') and results['documents'][0]:
    print(f"✅ Data is stored in ChromaDB. Found {len(results['documents'][0])} matching records.\n")
    for doc, metadata in zip(results['documents'][0], results['metadatas'][0]):
        print(f"🔹 **Law Type**: {metadata.get('law_type', 'Unknown')}")
        print(f"📜 **Section**: {metadata.get('section_summary', 'No summary available')}")
        print(f"🔗 **Source**: {metadata.get('source_url', 'No source available')}")
        print("-" * 50)
else:
    print("❌ No matching legal sections found in ChromaDB.")


✅ Data is stored in ChromaDB. Found 5 matching records.

🔹 **Law Type**: BNS
📜 **Section**: Section 101: IPC Section 100: Murder. Section 300: Culpable homicide. Section 400: Offending with the intention of causing bodily injury to any person. Section 500: Offence which is so dangerous that it must, in all probability, cause death or such bodily injury as is likely to cause death, and commits such act.
🔗 **Source**: https://devgan.in/bns/chapter_06.php
--------------------------------------------------
🔹 **Law Type**: BNS
📜 **Section**: Section 109: IPC section on murder and attempted murder. If hurt is caused by an act, the offender is liable to imprisonment for life or to such punishment as is hereinbefore mentioned. If a person offending under sub-section (1) is under sentence of imprisonment for. life, he may be punished with death.
🔗 **Source**: https://devgan.in/bns/chapter_06.php
--------------------------------------------------
🔹 **Law Type**: BNS
📜 **Section**: Section 104: W